In [ ]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets.mnist import MNIST
from torchvision.transforms import ToTensor

In [ ]:
# MNIST train 데이터를 다운로드 한다.
train_data = MNIST(root='./', train=True, download=True, transform=ToTensor())
# MNIST test 데이터를 다운로드 한다.
test_data = MNIST(root='./', train=False, download=True, transform=ToTensor())

In [ ]:
# train 데이터의 데이터 사이즈 확인
train_data.data.size()

In [ ]:
# target 값 확인
train_data.targets

In [ ]:
# 인덱스 1번째 이미지를 그려본다
plt.imshow(train_data.data[1], cmap='gray')
plt.show()

In [ ]:
# 데이터 로더로 배치 사이즈를 32로 설정한다.
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# 차수 확인
next(iter(train_loader))[0].shape

In [ ]:
# fmt : off
# 레이어 3개짜리 모델 생성 
model = nn.Sequential(
    nn.Linear(784, 64), 
    nn.ReLU(),
    nn.Linear(64, 32), 
    nn.ReLU(), 
    nn.Linear(32, 10)
)
# fmt : on
model

In [ ]:
# cuda 사용 가능하면 쿠다 사용 아니면 cpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'

device

In [ ]:
# 모델을 디바이스 메모리로 옮김
model.to(device)

In [ ]:
from torch.optim.adam import Adam

# 최적화 모델 아담 세팅 : 러닝레이트는 0.003
optim = Adam(model.parameters(), lr=1e-3)
# 로스값을 체크하기 위한 criterion 생성
criterion = nn.CrossEntropyLoss()
# 학습수 20번
epochs = 20

for epoch in range(epochs):
    # train loader에서 데이터와 레이블을 한개씩 가져옴
    for data, label in train_loader:
        # 가중치 값 초기화
        optim.zero_grad()
        # 데이터 차원수 변경 1차원으로
        data = torch.reshape(data, (-1, 784))
        # cuda 메모리에 옮림
        data = data.to(device)
        # 모델 평가
        pred = model(data)
        # 손실값 계산
        loss = criterion(pred, label.to(device))
        # 역전파 
        loss.backward()
        # 가중치값 옮김
        optim.step()

    # 손실값 1 에폭 마다 프린트
    print('epoch : ', loss)


In [ ]:
# 모델 추론
model.eval()

total = 0
correct = 0

# 기울기 계산 비활성
with torch.no_grad():
    # test loader에서 데이터 가져옴
    for data, label in test_loader:
        # 1차원으로 변환
        data = torch.reshape(data, (-1, 784))
        # cuda 메모리에 옮김
        data = data.to(device)
        # 평가
        pred = model(data)
        # 레이블값도 cuda 메모리에 옮김            
        label = label.to(device)

        # max 값 구함        
        _, predresult = torch.max(pred, 1)

        print(predresult)
        print(label)

        # 맞은 개수 구함
        correct += (predresult == label).sum().item()
        # 전체 개수 구함
        total += label.size(0)


In [ ]:
# 평균값
correct / total

In [ ]:
torch.max(torch.tensor([[1, 2, 3, 4, 5]]), 1)

In [ ]:
# 정답 확률
(predresult == label).sum() / 32

In [ ]:
label.size()